<center>
<img align="center" src="http://sydney.edu.au/images/content/about/logo-mono.jpg">
</center>
<h1 align="center" style="margin-top:10px">Statistical Learning and Data Mining</h1>
<h2 align="center" style="margin-top:20px">Week 6 Tutorial: Model Selection</h2>
<br>

We have two goals in this tutorial. The first is to illustrate how the concepts of overfitting, the bias-variance trade-off and the curse of dimensionality arise in practice.  The second is to discuss the practical implementation of cross-validation, hyperparameter optimisation and model stacking


<a href="#1.-Credit-card-data">Credit card data</a> <br>
<a href="#2.-Nested-cross-validation">Nested cross-validation</a> <br>
<a href="#3.-EDA-with-Lux">EDA with Lux</a> <br>
<a href="#3.-Interactive-data-visualisation-with-Plotly">Interactive data visualisation with Plotly</a> <br>
<a href="#4.-Feature-engineering">Feature engineering</a> <br>
<a href="#5.-Overfitting-and-the-bias-variance-trade-off">Overfitting and the bias-variance trade-off</a> <br>
<a href="#6.-Curse-of-dimensionality">Curse of dimensionality</a> <br>
<a href="#7.-Cross-validation">Cross-validation</a> <br>
<a href="#8.-Hyperparameter-optimisation-with-scikit-learn">Hyperparameter optimisation with scikit-learn</a> <br>
<a href="#9.-Hyperparameter-optimisation">Hyperparameter optimisation with optuna</a> <br>
<a href="#10.-Models">Models</a> <br>
<a href="#11.-Model-stacking">Model stacking</a> <br>
<a href="#12.-Validation-metrics">Validation metrics</a> <br>

In [9]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore') 

In [10]:
# Plot settings
sns.set_context('notebook') # optimises figures for notebook display
sns.set_style('ticks') # set default plot style
colours = ['#4E79A7','#F28E2C','#E15759','#76B7B2','#59A14F', 
          '#EDC949','#AF7AA1','#FF9DA7','#9C755F','#BAB0AB']
sns.set_palette(colours) # set custom color scheme
%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)

In [11]:
# Learning algorithms
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import mean_squared_error, r2_score,  mean_absolute_error

# Scaler
from sklearn.preprocessing import StandardScaler

# 1. Credit Card Data

In this tutorial, we will work with the `Credit` dataset from the  <a href="http://www-bcf.usc.edu/~gareth/ISL/index.html" target="_blank">ISL</a> textbook. The dataset records the average monthly credit card balance for customers of a bank, as well as other individual characteristics such age, education, gender, marital status, number of cards, and credit rating. 

Our objective is to predict the average monthly credit card balance of different clients.

In [12]:
data = pd.read_csv('Data/Credit.csv', index_col='Obs')
data.head()

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

# 2. Nested cross-validation

This is an advanced but necessary detail. We want to do achieve multiple goals in this tutorial:  

1. Select the number of neighbours and distance metric in the KNN method.
2. Train a model stack.
3. Compare the model stack with the individual models.  

Because hyperparameter optimisation and model stacking adapt the learning algorithms to the data, we need a second layer of validation or cross-validation to estimate the generalisation error of the entire machine learning pipeline.  Otherwise, the comparison would be biased in favour of methods that overfit the model selection criterion.

Therefore, we create a validation set for the specific purpose of comparing learning algorithms including hyperparameter optimisation and stacking. The hyperparameter optimisation and model stack will be based on cross-validation using the training set. 

In [13]:
from sklearn.model_selection import train_test_split

index_train, index_valid  = train_test_split(data.index, train_size=0.7, random_state=10)

# Write training and validation sets 
train = data.loc[index_train, :].copy() 
valid =  data.loc[index_valid, :].copy()

# Response label
response = 'Balance'

# Response vectors 
y_train = train[response]
y_valid = valid[response]

train.head()

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

# 3. EDA with Lux

[Lux](https://lux-api.readthedocs.io/en/latest/index.html) is another useful library for exploratory data analysis. It specialises on visual data exploration, unlike DataPrep and SweetViz, which build more comprehensive EDA reports.  

If everything went well with the installation, you should see a `Toggle Pandas/Lux` button above the dataframe. Click on it to see the visualisations. The information button explains what Lux did.  

In [14]:
import lux 
data = data.copy() # you need to create the dataframe after importing lux
data

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

Specifying the intent tells Lux that we're interested in a certain variable, in our case the response variable. 

We learn a few useful facts: 

* Many clients have a credit card balance of zero.
* The distribution of credit card balance is right skewed.
* Lux considered rating and limit to be the most "interesting" predictors (above, you can see that these two variables are actually nearly perfectly correlated).
* The relationship between balance and rating/limit is nonlinear. 

In [7]:
data.intent = ['Balance']
data

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

# 3. Interactive data visualisation with Plotly

Below, we will use the [Plotly](https://plotly.com/) package for interactive data visualisation. The code for some of the visualisations is a bit complicated, so I placed it in an auxiliary script file.

However, you can easily create standard plots such as scatter and line plots using Plotly express API. 

In [15]:
import plotly.express as px

fig = px.scatter(data, x='Limit', y='Balance', trendline='lowess')
fig.update_layout(template='plotly_white')
fig.show()

# 4. Feature engineering

We need to some quick feature engineering before continuing:

1. Dummy encoding for the nominal variables. 

2. Delete rating to avoid collinearity with limit. 

In [16]:
def feat_engineering(df): 
    df['Male']=(df['Gender'] ==' Male').astype(int) # creates dummy variable for gender
    df['Student']=(df['Student'] =='Yes').astype(int)
    df['Married']=(df['Married'] =='Yes').astype(int)
    df['Caucasian']=(df['Ethnicity'] =='Caucasian').astype(int)
    df['Asian']=(df['Ethnicity'] =='Asian').astype(int)
    df=df.loc[:, df.dtypes!='object'] # discards the columns that are not numerical
    df=df.drop(columns='Rating') # because of collinearity with limit
    return df
    
train = feat_engineering(train)
valid = feat_engineering(valid)

# 5. Overfitting and the bias variance trade-off

In the last tutorial, we established that the complexity of the KNN method depends on the number of neighbours. The lower the number of neighbours, the higher the model complexity.  

Let's focus on a single predictor (credit card limit) to illustrate how the behaviour of the learning algorithms changes with the number of neighbours. 

Use the slider to change the hyperparameter. 

**Activity: interpret the visualisation according to the lecture's discussion.**

In [17]:
%run -i tutorial6.py # loads the functions written in the tutorial6.py file

overfitting()

The problems of overfitting and underfitting arise because of the bias-variance trade-off. When model complexity is high, the learning algorithm has relatively low bias but high variance, which results in overfitting. When capacity is low, the learning algorithm has relatively low variance but high bias, which leads to underfitting.

The following figure plots the validation RMSE against the number of neighbours. 

**Activity: interpret the visualisation according to the lecture's discussion.**

In [18]:
biasvariance() # see the tutorial6.py file for the code

We can see that the validation RMSE has the U-shape predicted by the theory. When the number of neighbours is low, the validation RMSE is high because of overfitting. When the number of neighbours is high, the validation RMSE is high because of underfitting. Somewhere in between there is a number of neighbours that maximises predictive accuracy.

# 6. Curse of dimensionality

In the lectures, we discussed that the performance of examplar-based methods such as KNN does not scale well with number of predictors. 

We illustrate this problem as follows: 

1. Sort the predictors according to their correlation with the response. 

2. For both KNN and linear regression, compute validation predictions using an increasing number of inputs, starting from the one with highest correlation with the response and ending with the one that has the lowest. 

**Activity: interpret the visualisation according to the lecture's discussion.**

In [19]:
results = curse()  # see the tutorial6.py file for the code

# 7. Cross-validation

We use [cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html) to estimate the generalisation performance of the learning algorithm for purposes of model selection.

We'll use only two predictors in this section to facilitate visualisation.

In [20]:
predictors = ['Limit', 'Income'] 
X_train = train[predictors] 

<h3 style="padding-bottom: 10px">7.1 K-Fold Cross-Validation</h3>

The [<TT>cross_val_score</TT>](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function from Scikit-Learn allows you to compute the cross-validation error.

In [21]:
# Warning! This implementation assumes that the training rows are shuffled.  
# Our train-test split already did that 

from sklearn.model_selection import cross_val_score

knn = KNeighborsRegressor(n_neighbors=10, metric='mahalanobis', metric_params={'V': X_train.cov()})
scores = cross_val_score(knn, X_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
scores

array([-46938.3375    , -13504.56482143, -18188.13589286, -39258.8625    ,
       -13452.49964286])

The `cv=5` option specifies the number of folds, while `scoring = 'neg_mean_squared_error'` specifies the evaluation criterion. The function returns the score for each fold. Below, we average the scores and obtain the cross-validation root mean squared error. 

In [22]:
rmse = np.sqrt(-1*np.mean(scores))
print(rmse)

162.0755381648587


The scoring in Scikit-Learn follows the convention that higher score values are better. This is why the argument in the function is the negative mean squared error.  The scikit-learn [model evaluation](http://scikit-learn.org/stable/modules/model_evaluation.html) documentation provides a list of scoring options. It is useful to bookmark it for future reference. 

Let's apply what you've just learned plot the cross validation error as a function of the number of neighbours. 

In [23]:
kfold() # see the tutorial6.py file for the code

Using `k=6` neighbours leads to the lowest cross-validation error, but there are several other values between `k=5` and `k=11` lead to very similar performance.  

## 7.2 Repeated K-Fold

Because the size of the training data is not large, there is high variation in the MSE across folds. In turn, this indicates that the cross-validation error has large variance. In this case, it can be useful to use repeated k-fold to reduce the variance at the cost of additional computing time.

For this method we have to instantiate the cross-validation method separately. 

In [24]:
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=100)

knn = KNeighborsRegressor(n_neighbors=10, metric='mahalanobis', metric_params={'V': X_train.cov()})
scores = cross_val_score(knn, X_train, y_train, cv = rkf, scoring = 'neg_mean_squared_error')
np.sqrt(-1*np.mean(scores))  # five folds times ten repeats

158.11598563803003

In [25]:
repeatedkfold()

The curve now looks much smoother, which suggests that repeated k-fold successfully reduced the variance. We still select `k=6`, but the choice now seems more reliable. 

# 8. Hyperparameter optimisation with scikit-learn

## 8.1 Grid search

What we did in the last section was a manual grid search for `k`. Now, let's see how we can automate this process. 

In [32]:
from sklearn.model_selection import GridSearchCV
 
# Learning algorithm
model = KNeighborsRegressor(metric='mahalanobis', metric_params={'V': X_train.cov()}) 

# Hyperparameter grid
param_grid = {'n_neighbors': np.arange(1,51),}

# Run the grid search (cv=5 would do 5-fold CV instead)
knn_search =  GridSearchCV(model, param_grid, cv = rkf,  scoring = 'neg_mean_squared_error', verbose=1)
knn_search.fit(X_train, y_train)

Fitting 50 folds for each of 50 candidates, totalling 2500 fits


GridSearchCV(cv=RepeatedKFold(n_repeats=10, n_splits=5, random_state=100),
             estimator=KNeighborsRegressor(metric='mahalanobis',
                                           metric_params={'V':                Limit        Income
Limit   5.492703e+06  64999.556494
Income  6.499956e+04   1210.189671}),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
             scoring='neg_mean_squared_error', verbose=1)

The `best_params_` attribute allows us view the selected value of the hyperparameter.  

In [33]:
knn_search.best_params_

{'n_neighbors': 6}

The selected model is stored in the `best_estimator_` attribute: 

In [34]:
knn_search.best_estimator_

KNeighborsRegressor(metric='mahalanobis',
                    metric_params={'V':                Limit        Income
Limit   5.492703e+06  64999.556494
Income  6.499956e+04   1210.189671},
                    n_neighbors=6)

## 8.2 Randomised search

Later in the unit, we will study learning algorithms that have multiple hyperparameters, making a grid search prohibitively expensive. One option in this case is to perform a randomised search as follows.

This code is for illustrative purposes only: a randomised search is unnecessary here since we can quickly evaluate the full grid. 

In [35]:
from sklearn.model_selection import RandomizedSearchCV
 
# Learning algorithm
model = KNeighborsRegressor(metric='mahalanobis', metric_params={'V': X_train.cov()}) 

# Hyperparameter grid
param_grid = {'n_neighbors': np.arange(1,51),}

knn_search =  RandomizedSearchCV(model, param_grid, cv = rkf, n_iter=10, scoring = 'neg_mean_squared_error', 
                                 random_state=42)
knn_search.fit(X_train, y_train)

RandomizedSearchCV(cv=RepeatedKFold(n_repeats=10, n_splits=5, random_state=100),
                   estimator=KNeighborsRegressor(metric='mahalanobis',
                                                 metric_params={'V':                Limit        Income
Limit   5.492703e+06  64999.556494
Income  6.499956e+04   1210.189671}),
                   param_distributions={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
                   random_state=42, scoring='neg_mean_squared_error')

# 9. Hyperparameter optimisation with Optuna

We'll prefer [Optuna](https://optuna.readthedocs.io/en/stable/index.html) for hyperparameter optimisation in more advanced applications.  Optuna is very flexible, uses state-of-art methods, and works very well in practice.

Here's a preview of how it works. Optuna uses a Bayesian optimisation method by default. 

In [38]:
import optuna

# Create objective function
def objective(trial):

    # Suggest hyperparamter
    k = trial.suggest_int('k', 1, 51)
    
    # The rest is scikit-learn
    knn =  KNeighborsRegressor(n_neighbors=k , metric='mahalanobis', metric_params={'V': X_train.cov()})
    
    scores = cross_val_score(knn, X_train, y_train, cv = rkf, scoring = 'neg_mean_squared_error')
    rmse = np.sqrt(-1*np.mean(scores))
    
    return rmse

# Create and run study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, timeout=120) #timeout sets the maximum runtime

[I 2021-09-14 11:25:28,846] A new study created in memory with name: no-name-8a3e6105-0866-45e9-8ca9-a050518d0da6
[I 2021-09-14 11:25:29,018] Trial 0 finished with value: 170.75338839781753 and parameters: {'k': 21}. Best is trial 0 with value: 170.75338839781753.
[I 2021-09-14 11:25:29,187] Trial 1 finished with value: 176.45355763530412 and parameters: {'k': 27}. Best is trial 0 with value: 170.75338839781753.
[I 2021-09-14 11:25:29,362] Trial 2 finished with value: 198.9436981028983 and parameters: {'k': 50}. Best is trial 0 with value: 170.75338839781753.
[I 2021-09-14 11:25:29,531] Trial 3 finished with value: 176.45355763530412 and parameters: {'k': 27}. Best is trial 0 with value: 170.75338839781753.
[I 2021-09-14 11:25:29,700] Trial 4 finished with value: 172.9658758193292 and parameters: {'k': 23}. Best is trial 0 with value: 170.75338839781753.
[I 2021-09-14 11:25:29,866] Trial 5 finished with value: 157.81692444161888 and parameters: {'k': 9}. Best is trial 5 with value: 157

# 10. Models

Let's train full models now.

In [40]:
predictors = [x for x in train.columns if x != response]

print(predictors)

X_train  = train.loc[:,  predictors]
X_valid  = valid.loc[:,  predictors]

['Income', 'Limit', 'Cards', 'Age', 'Education', 'Student', 'Married', 'Male', 'Caucasian', 'Asian']


## 10.1 K-Nearest Neighbours

With kNN, we have the complication that it would not work well with all the predictors, as we've seen in the curse of dimensionality. Therefore, we modify the code to use the best combination of features.  We'll talk about feature selection next week. 

In [43]:
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector

subset = [0, 1 , 5] # Income, Limit, Student

model = make_pipeline(
    ColumnSelector(cols = subset),
    KNeighborsRegressor(metric='mahalanobis', metric_params={'V': train.iloc[:, subset].cov()})
    )

param_grid = {'kneighborsregressor__n_neighbors': np.arange(1, 51),}

knn =  GridSearchCV(model, param_grid, cv = rkf,  scoring = 'neg_mean_squared_error', verbose=1)
knn.fit(X_train, y_train)

knn.best_params_

Fitting 50 folds for each of 50 candidates, totalling 2500 fits


{'kneighborsregressor__n_neighbors': 3}

In [44]:
knn = make_pipeline(
    ColumnSelector(cols = subset),
    KNeighborsRegressor(n_neighbors=3, metric='mahalanobis', metric_params={'V': train.iloc[:, subset].cov()})
)

knn.fit(X_train, y_train)

Pipeline(steps=[('columnselector', ColumnSelector(cols=[0, 1, 5])),
                ('kneighborsregressor',
                 KNeighborsRegressor(metric='mahalanobis',
                                     metric_params={'V':                Income         Limit    Student
Income    1210.189671  6.499956e+04  -0.226186
Limit    64999.556494  5.492703e+06 -30.519713
Student     -0.226186 -3.051971e+01   0.081605},
                                     n_neighbors=3))])

## 10.2 Linear regression

Because the response is non-negative and has many zeros, a standard linear regression is not ideal for this problem. However, it's the tool that we have at moment, so let's use it. The tutorial practice will suggest some ways to get a better result.

In [45]:
ols = LinearRegression().fit(X_train, y_train)

 # 11. Model stack
 
We recommend that you build your final model by fitting a model average or more generally a model stack. A model average is a special case of a model stack and always a great place to start because it's useful to interpret the model weights. 

We use the [StackingCVRegressor](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.regressor/#stackingcvregressor) method from the `mlxtend` package. The meta-model will be a linear regression, which essentially results in a model average.

In [46]:
from mlxtend.regressor import StackingCVRegressor
 
stack = StackingCVRegressor(regressors=[ols, knn], meta_regressor=LinearRegression(), 
                           cv=5, random_state=1, store_train_meta_features=True)
stack.fit(X_train, y_train)

StackingCVRegressor(meta_regressor=LinearRegression(), random_state=1,
                    regressors=[LinearRegression(),
                                Pipeline(steps=[('columnselector',
                                                 ColumnSelector(cols=[0, 1,
                                                                      5])),
                                                ('kneighborsregressor',
                                                 KNeighborsRegressor(metric='mahalanobis',
                                                                     metric_params={'V':                Income         Limit    Student
Income    1210.189671  6.499956e+04  -0.226186
Limit    64999.556494  5.492703e+06 -30.519713
Student     -0.226186 -3.051971e+01   0.081605},
                                                                     n_neighbors=3))])],
                    store_train_meta_features=True)

Let's check the model weights. They do not sum to one exactly since we do not restrict the parameters, but we can interpret the result as saying that meta-model gives about 68% weight to kNN and 32% weight to the linear regression.

In [47]:
stack.meta_regr_.coef_

array([0.32474753, 0.70520681])

# 12. Validation metrics
 

In [48]:
# Initialise table
columns=['RMSE', 'R-Squared', 'MAE']
rows=['Linear Regression', 'KNN', 'Stack']
results =pd.DataFrame(0.0, columns=columns, index=rows)

# Methods
methods = [ols, knn, stack]

# Compute test predictions
y_pred = np.zeros((len(y_valid), 3))

for i, method in enumerate(methods):
    
    y_pred[:, i] = method.predict(X_valid)
    
# Validation results
for i in range(len(rows)):
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_valid, y_pred[:, i]))
    results.iloc[i, 1] = r2_score(y_valid, y_pred[:, i])
    results.iloc[i, 2] = mean_absolute_error(y_valid, y_pred[:, i])

results.round(3)

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()